<a href="https://colab.research.google.com/github/Stuti012/YouTube-Transcript-Summarizer/blob/main/YTsummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pytube -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00


In [2]:
from pytube import YouTube

In [3]:
VIDEO_URL = "https://www.youtube.com/watch?v=o0W_0MuvlwQ"

In [4]:
yt = YouTube(VIDEO_URL)

In [5]:
yt.streams
  .filter(only_audio = True, file_extension = 'mp4')
  .first()
  .download(filename = 'ytaudio.mp4')

'/content/ytaudio.mp4'

In [6]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Speech To Text(ASR)

In [7]:
!pip install huggingsound -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does 

In [8]:
from huggingsound import SpeechRecognitionModel

In [9]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda'

In [11]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

INFO:huggingsound.speech_recognition.model:Loading model...


#####To avoid out of memory, Audio Chunking

In [12]:
import librosa

In [13]:
input_file = '/content/ytaudio.wav'

In [14]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [15]:
import soundfile as sf

In [16]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

##ASR

In [17]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [18]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav']

In [19]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 6/6 [00:05<00:00,  1.13it/s]


In [20]:
full_transcript = ' '

In [21]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [22]:
len(full_transcript)

2090

In [23]:
full_transcript

" from morning coffee to afternoon tea caffine is so thoroughly entrenched in our daily routines and has become the world's most widely used psychoactive substancecaffine is a chemical compound that stimulates the central nervous systemit accomplishes this by attaching itself to adennicine receptors in the brainin doing so it blocks a dennocine moleculules which are responsible for slowing down the central nervous system before wesleepthe result a denicine cannot regulate and nurons start firingthis results in a person's reduced fatigue and increased alertness and cognitive performance side-effects many seek by consuming caffeenecaffeen occurs naturally in coffee beans tealeaves ccowbeans and some treenuts people have been consuming it firstcenturies but daily consumption took off sporadically at different times in different places around the worldcoffee became popular in the fifteenth and sixteenth centuries but gakau was used among early mezo-american civilizations hundreds of years 

#Summarizer

In [24]:
from transformers import pipeline

In [25]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [26]:
summarized_text = summarization(full_transcript)

Below is the summarization of the given youtube video

In [27]:
summarized_text[0]['summary_text']

' Caffine is a chemical compound that stimulates the central nervous system . The result is reduced fatigue and increased alertness and cognitive performance side-effects many seek by consuming caffeen . In the united states nearly ninety percent of the population has at least one cafinated beverage every day .'

####Text chunking before summarization

Below code summarizes a chunk of text from given text file.

In [28]:
num_iters = int(len(full_transcript)/1000) #cutting text into 1000 characters and for every 1000 ch summarization is done.
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
  print("Summarized text\n"+out)
  summarized_text.append(out)

print(summarized_text)

input text 
 from morning coffee to afternoon tea caffine is so thoroughly entrenched in our daily routines and has become the world's most widely used psychoactive substancecaffine is a chemical compound that stimulates the central nervous systemit accomplishes this by attaching itself to adennicine receptors in the brainin doing so it blocks a dennocine moleculules which are responsible for slowing down the central nervous system before wesleepthe result a denicine cannot regulate and nurons start firingthis results in a person's reduced fatigue and increased alertness and cognitive performance side-effects many seek by consuming caffeenecaffeen occurs naturally in coffee beans tealeaves ccowbeans and some treenuts people have been consuming it firstcenturies but daily consumption took off sporadically at different times in different places around the worldcoffee became popular in the fifteenth and sixteenth centuries but gakau was used among early mezo-american civilizations hundred